In [1]:
#1. 모듈 호출
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [2]:
#2. 사용자 인터페이스 만들기 
print('='*100)
print('네이버 뉴스기사 댓글 크롤러')
print('='*100)
url = input('1.댓글을 크롤링할 뉴스 URL을 입력하세요 : ')
key_word2 = '네이버 뉴스 기사 댓글자료 '
cnt = int(input('2.크롤링 할 건수는 몇건입니까? : '))
f_dir = input('3.파일을 저장할 폴더명만 쓰세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word2)
os.chdir(f_dir+s+'-'+key_word2)

f_name = s+'-'+key_word2+'.txt'
fc_name = s+'-'+key_word2+'.csv'
fx_name = s+'-'+key_word2+'.xlsx'

s_time = time.time()

네이버 뉴스기사 댓글 크롤러
1.댓글을 크롤링할 뉴스 URL을 입력하세요 : https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012174703&isYeonhapFlash=Y&rc=N
2.크롤링 할 건수는 몇건입니까? : 100
3.파일을 저장할 폴더명만 쓰세요 : 


In [3]:
#3. 웹드라이버 호출 
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

driver = webdriver.Chrome(path, options=options)
driver.get(url)
time.sleep(random.randrange(2,5))

In [4]:
driver.find_element_by_class_name('lo_txt').click()
time.sleep(3)

driver.find_element_by_tag_name('body').send_keys(Keys.END)
time.sleep(2)

In [5]:
if cnt <= 20 : 
    html0 = driver.page_source 
    soup0 = BeautifulSoup(html0, 'html.parser')
    div_list0 = soup0.find('ul', class_='u_cbox_list').find_all('div', 'u_cbox_area')
    
no22 = 2    
if cnt > 20 : 
    while no22 == 2 : 
        try : 
            driver.find_element_by_class_name('u_cbox_btn_more').click()
            time.sleep(2)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(2)
        
            html0 = driver.page_source 
            soup0 = BeautifulSoup(html0, 'html.parser')
            div_list0 = soup0.find('ul', class_='u_cbox_list').find_all('div', 'u_cbox_area')
            if len(div_list0) >= cnt : 
                break
        except : 
            print('%s 건 댓글 수집 요청하셨으나, %s 건 밖에 존재하지 않아서 %s 건 만큼만 수집하겠습니다' %(cnt, len(div_list0), len(div_list0)))
            cnt = len(div_list0)
            break
            

In [6]:
nick1 = []
cont1 = []
date1 = []
go1 = []
bgo1 = []
no11 = 1

f = open(f_name, 'a', encoding='utf-8')
for a in div_list0 : 
    print('%s 번째 댓글 수집 중==================' %no11)
    f.write('%s 번째 댓글 수집 중==============' %no11 + '\n')
    
    nick = a.find('span', class_='u_cbox_name_area').get_text().replace('\n', "").strip()
    print('1.작성자ID :', nick)
    f.write('1.작성자ID :'+nick+'\n')
    nick1.append(nick)
    
    try : 
        review = a.find('span', class_='u_cbox_contents').get_text().replace('\n', "").strip()
        print('2.댓글내용 :',review)
        f.write('2.댓글내용 :'+review+'\n')
        cont1.append(review)
    except : 
        try : 
            review = a.find('span', class_='u_cbox_cleanbot_contents').get_text().replace('\n', "").strip()
            print('2.댓글내용 :',review)
            f.write('2.댓글내용 :'+review+'\n')
            cont1.append(review)
        except : 
            f.write('2.댓글내용 : 작성자가 삭제한 댓글 입니다'+'\n')
            print('2. 댓글내용 : 작성자가 삭제한 댓글 입니다')
            cont1.append("작성자가 삭제한 댓글 입니다")
    
    date = a.find('span', class_='u_cbox_date').get_text().replace('\n', "").strip()
    print('3.작성일자 :',date)
    f.write('3.작성일자 :'+date+'\n')
    date1.append(date)
    
    try : 
        gogam = a.find('em', class_='u_cbox_cnt_recomm').get_text().replace('\n', "").strip()
        print('4.공감횟수 :',gogam)
        f.write('4.공감횟수 :'+gogam+'\n')
        go1.append(gogam)
    except : 
        f.write('4. 공감횟수 : 0'+'\n')
        print('4. 공감횟수 : 0')
        go1.append('0')
    
    try : 
        bgogam = a.find('em', class_='u_cbox_cnt_unrecomm').get_text().replace('\n', "").strip()
        print('5.비공감횟수 : ',bgogam)
        f.write('5.비공감횟수 :'+bgogam+'\n')
        bgo1.append(bgogam)
    except : 
        f.write('5.비공감횟수 : 0'+'\n')
        print('5.비공감횟수 : 0')
        bgo1.append('0')
    
    f.write('%s 건 완료 =====================' %no11 + '\n')
    f.write('\n')
    print('%s 건 완료 =================' %no11)
    print('\n')
    
    no11 += 1 
    if no11 > cnt : 
        break
            
driver.quit()
f.close()
e_time = time.time()

1 번째 댓글 수집 중==================
1.작성자ID : mrso****
2.댓글내용 : 참 꼼꼼도 하셔라.. 애초에 싱가포르나 대만처럼  해외유입 차단했으면 자영업자들 이렇게까지 🐕 고생은 안했을텐데..ㅉ
3.작성일자 : 2021.02.01. 05:13
4.공감횟수 : 597
5.비공감횟수 :  50
1 건 완료 =================


2 번째 댓글 수집 중==================
1.작성자ID : piok****
2.댓글내용 : 국민들아.....재난지원금 받는다고 이번선거에 현혹되지 말거라....받을건 모두 다 챙기고....이번 시장선거만큼은 더러운민주출신은 절대 안됩니다....무조건 다른당에 투표하시고.....인물 없으니....따지지마시고....이번투표는 "당" 을 바꾸는 것이 "팩트 " 입니다....명심하세요...
3.작성일자 : 2021.02.01. 05:33
4.공감횟수 : 568
5.비공감횟수 :  77
2 건 완료 =================


3 번째 댓글 수집 중==================
1.작성자ID : don0****
2.댓글내용 : 그냥 사람들이 모이는게 두려운거야 이 정권은.....대형백화점에는 사람들이 바글바글 스키장에도 바글바글 뭐 거긴 식당이 없냐? 다들 마스크 안벗고 먹어? 그리고 요즘 식당들 보면 9시 이후에 영업금지라 오히려 9시이전에 사람들이 더 와글와글 모이더라? 9시 이후에는 바이러스가 더 전파력이 강해지는거냐?...9시 이후에 식당금지.....뭐 새로운 유형의 통행금지 같은거냐?
3.작성일자 : 2021.02.01. 05:50
4.공감횟수 : 297
5.비공감횟수 :  16
3 건 완료 =================


4 번째 댓글 수집 중==================
1.작성자ID : madu****
2.댓글내용 : 뭔가 전체적인 방향성을 그려 놓고 운영을 하는게 아니라 논리와 명분없이 야금야금 하나씩 던져주는 느낌이다. 스키장 9시 이후 영업재개해주고 셔틀금

In [7]:
total_time = e_time - s_time 

news = pd.DataFrame()
news['작성자ID'] = nick1
news['작성내용'] = cont1
news['작성일자'] = date1
news['공감횟수'] = go1
news['비공감횟수'] = bgo1

news.to_csv(fc_name, encoding='utf-8-sig')
news.to_excel(fx_name)

print('='*100)
print('총 %s 건 댓글 수집했습니다' %cnt)
print('총 소요시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)

총 100 건 댓글 수집했습니다
총 소요시간은 30.3 초 입니다


118